ユニット配列のクラスタリングを、split-merge sampling によって行う。

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.display import display
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.io as pio
pio.templates.default = 'plotly_white'
import logging
import logzero
logzero.loglevel(logging.DEBUG)

In [2]:
dir_fname = 'work'
import os
os.chdir(dir_fname)

In [3]:
import numpy as np
from copy import copy
import pandas as pd
from BITS.clustering.seq import ClusteringSeq
from BITS.seq.align import EdlibRunner
from BITS.seq.consed import ConsedRunner
from BITS.util.io import save_pickle, load_pickle
from BITS.util.io import load_pickle, save_pickle
import numpy as np
import pandas as pd
from BITS.plot.plotly import make_hist, make_scatter, make_layout, show_plot
from BITS.clustering.seq import ClusteringSeq
import consed
from BITS.seq.align import EdlibRunner
from collections import Counter, defaultdict
from logzero import logger
from dataclasses import dataclass
from typing import List
import random
from copy import copy
from collections import Counter
from dataclasses import dataclass
from logzero import logger
import numpy as np
from vca.types import TRUnit
from BITS.clustering.seq import ClusteringSeq
from BITS.seq.align import EdlibRunner
from BITS.util.io import load_pickle, save_pickle
from BITS.util.proc import NoDaemonPool
from BITS.seq.utils import revcomp

In [4]:
def read_to_forward_units(read):
    """Return all units contained in <read> so that starnd of every unit is same as the forward master unit."""
    return [read.seq[unit.start:unit.end] if unit.id == 0 else revcomp(read.seq[unit.start:unit.end])
            for unit in read.units]

In [5]:
sync_reads = load_pickle("centromere_reads_incl_low_cover_rate_all_sync.pkl")

In [43]:
len(sync_reads)

1905

In [8]:
sync_units = []
for read in sync_reads:
    sync_units += read_to_forward_units(read)

In [9]:
len(sync_units)

46538

In [6]:
centromere_phreds = load_pickle("centromere_phreds_incl_low_cover_rate_all_sync.pkl")

In [ ]:
sync_quals = []
for read in sync_reads:
    sync_quals += [centromere_phreds[read.name][unit.start:unit.end] if unit.id == 0
                   else np.flip(centromere_phreds[read.name][unit.start:unit.end])
                   for unit in read.units]

-> この数ではクラスタリングも厳しい。

In [7]:
db_prefix = "DMEL_CSS"
db_fname = f"{db_prefix}.db"
las_fname = f"TAN.{db_prefix}.las"
from vca import ReadViewer
v = ReadViewer(db_fname, las_fname)

In [8]:
sync_reads_by_id = {read.id: read for read in sync_reads}

In [8]:
v.show(read=sync_reads_by_id[73])

[D 190920 01:19:32 find_units:90] inners: {SelfAlignment(ab=354, ae=13802, bb=3, be=13447)}
[I 190920 01:19:32 log:17] Starting distance matrix calculation 
[D 190920 01:19:32 seq:19] Computing row 0
[D 190920 01:19:32 seq:19] Computing row 36
[D 190920 01:19:32 seq:19] Computing row 1
[D 190920 01:19:32 seq:19] Computing row 35
[D 190920 01:19:32 seq:19] Computing row 2
[D 190920 01:19:32 seq:19] Computing row 34
[D 190920 01:19:32 seq:19] Computing row 3
[D 190920 01:19:32 seq:19] Computing row 33
[D 190920 01:19:32 seq:19] Computing row 4
[D 190920 01:19:32 seq:19] Computing row 32
[D 190920 01:19:32 seq:19] Computing row 5
[D 190920 01:19:32 seq:19] Computing row 31
[D 190920 01:19:32 seq:19] Computing row 6
[D 190920 01:19:32 seq:19] Computing row 30
[D 190920 01:19:32 seq:19] Computing row 7
[D 190920 01:19:32 seq:19] Computing row 29
[D 190920 01:19:32 seq:19] Computing row 8
[D 190920 01:19:32 seq:19] Computing row 28
[D 190920 01:19:32 seq:19] Computing row 9
[D 190920 01:19:3

In [28]:
sync_reads_by_id[762]

TRRead(seq='aaatccttcaaaaagtgatagtaatcgttagcgctggtaataagctgttaaaaaccgttattcttacatctatatgaccattttagccgagtaatgacgaaaattccgattgtaaatatcaacttttaggcaaaatccgtttttccaaatttcggtcatcaaataatcagttttttttttttgccacaactttataaatatttgtctgaatacggaatgtcatacttcgctgagctcgtaattacatttacaattaaaatgtgttcaaaaatggaaattatgtttttttgccatacttttcaaatttttatggggggtcattacaaaaaatacgaaaatttacccaaaaattgatttccctaaatccttcaataagtgatagtaatcgttagcgctggtaataagctgttaaaaaccgttattcttacatctatatgaccattttagccgagtaatgacgaaaattccgtttgtaaatatcaactttttggcaaaatccgtttttccaaatttcggtcatcaaataatcagttttttttttgccacaactttataaatatttgtctgaatacggaatgtcatacctcgctgagctcgtaattacatttaaaattaaaatgtgttcaaaaatggaaattatgttttttgccatactttgcaaatttttatggggggtcattacaaaaaatacgaaatttgacccaaatattgatttccctaaatccttcaaaaagtgatagtaatcgttagcgctggtaataagctgttaaaaaccgttattcttacatctatatgaccattttagccgagtaatgacgaaaattccgattgtaaatatcaacttttaggcaaaatccgttttttccaaatttcggtcatcaaataatcagtttttttttttttgccacaactttataaatatttgtctgaatacggaatgtcatacttcgctgagctcgtaattacatttacaattaaaatgtgttca

In [78]:
!(cat master_unit)

caaaaatgttgatatttacaaacgaaattttcgttataacttggctaaaaatggtcacatagatgtaagaataactgttttgagcagctaattaccagtgctaacgatccctattactttttgaaggatttagggaaattaatttttggatcaattttcgcattttttgtaaggaggggggtcatcaaaatttgcaaaatatggccaaaaaatttaatttccatttttgaacacagtttgattggaaattttattacgagctcagtgaggtatgacattccatattcagacaattattttttatgttgtggcaaaataaatgattatttgatgaccgaaatttggaaaaacagactctg


In [9]:
master_unit = "caaaaatgttgatatttacaaacgaaattttcgttataacttggctaaaaatggtcacatagatgtaagaataactgttttgagcagctaattaccagtgctaacgatccctattactttttgaaggatttagggaaattaatttttggatcaattttcgcattttttgtaaggaggggggtcatcaaaatttgcaaaatatggccaaaaaatttaatttccatttttgaacacagtttgattggaaattttattacgagctcagtgaggtatgacattccatattcagacaattattttttatgttgtggcaaaataaatgattatttgatgaccgaaatttggaaaaacagactctg"

## Functions for counting variants

In [9]:
# Variants and sequencing errors

class PairwiseAlignment:
    def __init__(self, a_seq, b_seq):
        er = EdlibRunner("global", revcomp=False, cyclic=False)
        self.fcigar = er.align(b_seq.lower(), a_seq.lower()).cigar.flatten().string   # NOTE: b vs a; be careful!
        self.source, self.target = '', ''
        s_pos, t_pos = 0, 0
        for c in self.fcigar:
            if c == '=' or c == 'X':
                self.source += a_seq[s_pos]
                self.target += b_seq[t_pos]
                s_pos += 1
                t_pos += 1
            elif c == 'I':
                self.source += '-'
                self.target += b_seq[t_pos]
                t_pos += 1
            else:
                self.source += a_seq[s_pos]
                self.target += '-'
                s_pos += 1
        
    def show(self, by_cigar=False):
        if by_cigar:   # standard alignment like BLAST
            print(self.source)
            print(self.fcigar)
            print(self.target)
        else:
            print(''.join([' ' if c == '=' else self.source[i] for i, c in enumerate(self.fcigar)]))
            print(''.join([self.source[i] if c == '=' else ' ' for i, c in enumerate(self.fcigar)]))
            print(''.join([' ' if c == '=' else self.target[i] for i, c in enumerate(self.fcigar)]))

def count_variants(cluster_cons_unit, cluster_units):
    """Given a set of unit sequences <units> in a cluster, calculate the composition of
    nucleotides including '-' (= distribution of each )
    for each position on <cluster_cons_unit> as a seed.
    from which <units> are generated, compute the variations (= nucleotides inconsistent between
    <units> and <cluster_cons_unit> and their relative frequency).
    Since a cluster should be homogeneous (i.e., mono-source), the relative frequencies are
    expected to be not much larger than sequencing error.
    """
    assert cluster_cons_unit != "", "Empty strings are not allowed"
    # TODO: how to decide "same variant?" especially for multiple variations on same position (but slightly different among units)?
    variants = Counter()
    for unit in cluster_units:
        assert unit != "", "Empty strings are not allowed"
        alignment = PairwiseAlignment(cluster_cons_unit, unit)   # alignment.fcigar(cluster_cons_unit) = unit
        tpos = 0
        var_index = 0   # positive values for continuous insertions
        for i, c in enumerate(alignment.fcigar):
            if c == '=':
                var_index = 0
            elif c == 'I':
                var_index += 1
            if c != '=':
                variants[(tpos, var_index, c, alignment.target[i])] += 1   # TODO: multiple D on the same pos are aggregated
            if c != 'I':
                tpos += 1
        assert tpos == len(cluster_cons_unit)
    return variants

def list_variations(template_unit, cluster_cons_unit):
    """Single-vs-single version of count_variants().
    That is, list up the differences between the (imaginary) template unit and the consensus unit
    of a cluster (which should be a real instance).
    The return value is [(position_on_template_unit, variant_type, base_on_cluster_cons_unit)].
    """
    assert template_unit != "" and cluster_cons_unit != "", "Empty strings are not allowed"
    return list(count_variants(template_unit, [cluster_cons_unit]).keys())

## Functions for alignment probability

In [10]:
# Probability of alignment 1) between units and 2) between a unit and a representative unit

def phred_to_log_p_correct(phred):
    return np.log10(1 - np.power(10, -phred / 10))

def phred_to_log_p_error(phred):
    return -phred / 10

def log_prob_gen(cons_unit, obs_unit, obs_qual=None, p_non_match=0.01):
    """Log likelihood of generating <obs_unit> from <cons_unit>.
    <obs_qual> is positional QVs of <obs_unit> and if not given,
    <p_non_match> is used as average error rate for every position.
    """
    if cons_unit == "":   # input sequences for <cons_unit> were empty; or, Consed did not return
        return -np.inf

    # Compute alignment
    er = EdlibRunner("global", revcomp=False)
    fcigar = er.align(cons_unit, obs_unit).cigar.flatten().string
    #logger.debug(fcigar)
    
    # Calculate the sum of log probabilities for each position in the alignment
    if obs_qual is None:
        n_match = Counter(fcigar)['=']
        n_non_match = len(fcigar) - n_match
        return n_match * np.log10(1 - p_non_match) + n_non_match * np.log10(p_non_match)
    else:
        p = 0.
        pos = 0
        for c in fcigar:
            p += (phred_to_log_p_correct(obs_qual[pos]) if c == '='
                  else phred_to_log_p_error(obs_qual[pos]))
            if c in ('=', 'X', 'D'):
                pos += 1
        assert pos == len(obs_unit) == len(obs_qual), "Invalid length"
        return p

def log_prob_align(unit_x, unit_y, qual_x=None, qual_y=None, p_error=0.01):
    """Log likelihood of alignment between <unit_x> from <unit_y>.
    <qual_*> is positional QVs of <unit_*> and if not given,
    <p_error> is used as average error rate for every position of each read.
    """
    # Compute alignment
    er = EdlibRunner("global", revcomp=False)
    fcigar = er.align(unit_x, unit_y).cigar.flatten().string
    #logger.debug(fcigar)
    
    # Calculate the sum of log probabilities for each position in the alignment
    if qual_x is None and qual_y is None:
        p_match = (1 - p_error) * (1 - p_error)
        n_match = Counter(fcigar)['=']
        n_non_match = len(fcigar) - n_match
        return n_match * np.log10(p_match) + n_non_match * np.log10(1 - p_match)
    else:
        p = 0.
        pos_x = pos_y = 0
        for c in fcigar:   # fcigar(unit_y) = unit_x
            p_match = phred_to_log_p_correct(qual_x[pos_x]) + phred_to_log_p_correct(qual_y[pos_y])
            p += (p_match if c == '='
                  else np.log10(1 - np.power(10, p_match)))
            if c in ('=', 'X', 'I'):
                pos_x += 1
            if c in ('=', 'X', 'D'):
                pos_y += 1
        assert pos_x == len(unit_x) == len(qual_x) and pos_y == len(unit_y) == len(qual_y), "Invalid length"
        return p
    
def log_factorial(n):
    return np.sum([np.log10(i) for i in range(1, n + 1)])
    
def log_prob_composition(cons_unit, obs_units, p_error=0.001):   # TODO: use positional QVs
    """Log likelihood of composition of <obs_units> given <cons_unit> as a seed; i.e., probability of alignment pileup.
    <p_error> is used for average sequencing error rate (= non-match rate in a single alignment).
    Concretely, compute Multinomial(n_A, ..., n_-; p_A, ..., p_-) for each position, where p_X = 1 - p_error
    if X is the base of <cons_unit>, otherwise p_X = p_error.
    """
    var_counts = count_variants(cons_unit, obs_units)
    var_pos = [pos for pos, index, op, base in var_counts.keys()]
    
    # compute for matches
    n_matches = len(cons_unit) - len(set(var_pos))
    p_match = n_matches * len(obs_units) * np.log10(1 - p_error)
    
    # compute for variants
    var_freqs = defaultdict(Counter)   # {(pos, index): Counter('A': n_A, ..., '-': n_-)} for each variant column
    for (pos, index, op, base), count in var_counts.items():   # list up frequencies of each variant for each position
        var_freqs[(pos, index)][base] = count
    log_factorial_N = log_factorial(len(obs_units))
    p_var = 0.
    for key, counts in var_freqs.items():   # for each variant position
        p_var += log_factorial_N
        for base, count in counts.items():
            p_var -= log_factorial(count)
            p_var += count * np.log10(p_error)
        n_match = len(obs_units) - np.sum(list(counts.values()))   # number of units having base same as seed
        p_var -= log_factorial(n_match)
        p_var += n_match * np.log10(1 - p_error)

    return p_match + p_var

## Split-merge clustering

In [11]:
def normalize_assignments(assignments):
    """Convert a clustering state <assignments> so that all essentially equal states can be
    same array. Return value type is Tuple so that it can be hashed as a dict key.
    """
    convert_table = {}
    new_id = 0
    for cluster_id in assignments:
        if cluster_id not in convert_table:
            convert_table[cluster_id] = new_id
            new_id += 1
    return tuple([convert_table[cluster_id] for cluster_id in assignments])

@dataclass(eq=False)
class SplitMergeClustering:
    units: List[str]
    quals: np.ndarray
    alpha: float
    
    def __post_init__(self):
        self.N = len(self.units)   # number of data
        self.assignments = np.zeros([self.N], dtype=np.int16)   # cluster assignments

        # Compute all-vs-all unit alignment likelihood
        #self.log_p_mat = np.zeros([self.N, self.N], dtype=np.float32)
        #for i in range(self.N):
        #    for j in range(i + 1, self.N):
        #        self.log_p_mat[i][j] = self.log_p_mat[j][i] = log_prob_align(self.units[i], self.units[j],
        #                                                                    self.quals[i], self.quals[j])
        
        # Cache for values computationally expensive
        self.cache_log_prob_clustering = {}   # {normalized_assignments: probability}
        self.cache_cluster_cons = {}   # {unit_ids: cluster_cons}
        
        # Pre-compute some constants
        self.const_ewens = -np.sum([np.log10(self.alpha + i) for i in range(self.N)])
        self.const_gibbs = -np.log10(self.N - 1 + self.alpha)
        
        # Compute consensus unit of the whole units so that comparing clusters can be easy
        self.template_unit = self.cluster_cons(0)
        
    def show_clustering(self):
        er = EdlibRunner("global", revcomp=False)
        for cluster_id in np.unique(self.assignments):
            print(f"Cluster {cluster_id} ({len(self.cluster_units(cluster_id))} units):\n"
                  f"{self.cluster_unit_ids(cluster_id)}\n"
                  f"{self.cluster_cons(cluster_id)}\n"
                  f"{er.align(self.cluster_cons(cluster_id), self.template_unit).cigar.flatten().string}")
            print("---")
            for unit in self.cluster_units(cluster_id):
                print(f"{er.align(unit, self.cluster_cons(cluster_id)).cigar.flatten().string}")
                
    def n_units(self, cluster_id, assignments=None, exclude_unit=None):
        """Return the number of units in the cluster <cluster_id> given a clustering state <assignments>,
        while excluding a unit <exclude_unit> if provided."""
        return len(self.cluster_unit_ids(cluster_id, assignments, exclude_unit))
            
    def cluster_unit_ids(self, cluster_id, assignments=None, exclude_unit=None):
        """Return indices of the units belonging to the cluster <cluster_id> given a clustering state <assignments>,
        while excluding a unit <exclude_unit> if provided."""
        return [i for i in range(self.N)
                if i != exclude_unit and (self.assignments if assignments is None else assignments)[i] == cluster_id]

    def cluster_units(self, cluster_id, assignments=None, exclude_unit=None):
        """Return unit sequences belonging to the cluster <cluster_id> given a clustering state <assignments>,
        while excluding a unit <exclude_unit> if provided."""
        return [self.units[i] for i in self.cluster_unit_ids(cluster_id, assignments, exclude_unit)]
    
    def n_clusters(self, assignments=None):
        """Return the number of clusters."""
        return len(self.cluster_ids(self.assignments if assignments is None else assignments))

    def cluster_ids(self, assignments=None):
        """Return a list of cluster indices."""
        return np.unique(self.assignments if assignments is None else assignments)

    def cluster_cons(self, cluster_id, assignments=None, exclude_unit=None):
        """Return the consensus sequence of the units belonging to the cluster <cluster_id> given a clustering state <assignments>,
        while excluding a unit <exclude_unit> if provided."""
        # Check the cache
        cluster_units = set(self.cluster_units(cluster_id, assignments))
        excluded = False
        if exclude_unit is None or exclude_unit not in cluster_units:
            if tuple(sorted(cluster_units)) in self.cache_cluster_cons:
                return self.cache_cluster_cons[tuple(sorted(cluster_units))]
        else:
            cluster_units.remove(exclude_unit)
            excluded = True
        cluster_units = sorted(cluster_units)
        
        #cluster_units = self.cluster_units(cluster_id, assignments, exclude_unit)   # units belonging to the cluster
        if len(cluster_units) == 0:   # cluster with single unit which is excluded
            cons = ""
        elif len(cluster_units) == 1:   # cluster with single unit
            cons = cluster_units[0]   # TODO: NOTE: single data cluster can be harmful!
        else:
            cons = consed.consensus(cluster_units)
        
        if not excluded:
            self.cache_cluster_cons[tuple(cluster_units)] = cons
        return cons
        
    def log_prob_ewens(self, assignments=None):
        """Return the probability of partition."""
        p = self.n_clusters() * np.log10(self.alpha)
        for cluster_id in self.cluster_ids(assignments):
            p += log_factorial(self.n_units(cluster_id, assignments) - 1)
        return p + self.const_ewens
    
    def log_prob_cluster_composition(self, cluster_id, assignments=None, p_error=0.001):
        """Return log probability of the composition of the cluster <cluster_id> given a clustering state <assignments>"""
        return log_prob_composition(self.cluster_cons(cluster_id, assignments),
                                    self.cluster_units(cluster_id, assignments))

    def log_prob_units_generation(self, cluster_id, assignments=None):
        """Return log probability of generating the units belonging to a cluster <cluster_id> from the cluster
        given a clustering state <assignments>."""
        cons = self.cluster_cons(cluster_id, assignments)
        return np.sum([log_prob_gen(cons, unit) for unit in self.cluster_units(cluster_id, assignments)])
        
    def log_prob_clustering(self, assignments=None):
        """Compute the joint probability of the current clustering state."""
        # Check the cache
        normalized_assignments = normalize_assignments(self.assignments if assignments is None else assignments)
        if normalized_assignments in self.cache_log_prob_clustering:
            logger.debug(f"Found in cache")
            return self.cache_log_prob_clustering[normalized_assignments]
        
        # First of all, check if consensus sequence exists for every cluster
        for cluster_id in self.cluster_ids(assignments):
            cons = self.cluster_cons(cluster_id, assignments)
            if cons == "":   # Consed did not return
                return -np.inf

        p_ewens = self.log_prob_ewens(assignments)
        p_cluster_compositions = np.sum([self.log_prob_cluster_composition(cluster_id, assignments)
                                         for cluster_id in self.cluster_ids(assignments)])
        p_gen_units = np.sum([self.log_prob_units_generation(cluster_id, assignments)
                              for cluster_id in self.cluster_ids(assignments)])

        p = p_ewens + p_cluster_compositions + p_gen_units
        self.cache_log_prob_clustering[normalized_assignments] = p
        return p
        
    def gibbs_sampling_single(self, unit_id, cluster_ids, assignments):
        """Compute probability of the unit assignment for each cluster while excluding the unit."""
        # NOTE: here assignment to a new cluster is not considered because of its very low probability
        #weights = tuple(map(lambda log_p: np.power(10, log_p),
        #                    [(np.log10(self.n_units(cluster_id, assignments, exclude_unit=unit_id))
        #                      - np.log10(self.N - 1 + self.alpha)
        #                      + log_prob_gen(self.cluster_cons(cluster_id, assignments, exclude_unit=unit_id),
        #                                  self.units[unit_id]))
        #                     for cluster_id in cluster_ids]))
        #new_assignment = random.choices(cluster_ids, weights=weights)[0]   # sample a new cluster assignment based on the probabilities
        #logger.debug(f"weights: {weights}, {assignments[unit_id]} -> {new_assignment}")
        #return new_assignment
        
        # NOTE: below is a proxy of Gibbs sampling; deterministically decide the nearest cluster as assignment
        max_prob = 0.
        max_cluster_id = -1
        for cluster_id in cluster_ids:
            log_p = (np.log10(self.n_units(cluster_id, assignments, exclude_unit=unit_id))
                     + self.const_gibbs
                     + log_prob_gen(self.cluster_cons(cluster_id, assignments, exclude_unit=unit_id),
                                    self.units[unit_id]))
            p = np.power(10, log_p)
            if p > max_prob:
                max_prob = p
                max_cluster_id = cluster_id
        return max_cluster_id

    def gibbs_sampling(self, unit_ids, cluster_ids, assignments, n_iter=1):
        """Re-assign each unit of <unit_ids> into one of the clusters <cluster_ids>,
        Given a clustering state <assignments>.
        """
        for t in range(n_iter):
            logger.debug(f"Round {t}")
            for unit_id in unit_ids:
                #logger.debug(assignments)
                old_assignment = assignments[unit_id]
                assignments[unit_id] = self.gibbs_sampling_single(unit_id, cluster_ids, assignments)
                new_assignment = assignments[unit_id]
                logger.debug(f"Unit {unit_id}: {old_assignment} -> {new_assignment}")
        return assignments
    
    def do_gibbs(self, n_iter=3):
        """Run a single iteration of Gibbs sampling with all units."""
        p_old = self.log_prob_clustering()
        self.assignments = self.gibbs_sampling(list(range(self.N)), self.cluster_ids(), self.assignments, n_iter)
        p_new = self.log_prob_clustering()
        logger.debug(f"State prob: {p_old} -> {p_new}")
        logger.debug(self.assignments)
    
    def do_proposal(self, n_iter=30):
        """Propose a new state by choosing random two units."""
        for t in range(n_iter):
            x, y = random.sample(list(range(self.N)), 2)
            logger.debug(f"Selected: {x}({self.assignments[x]}) and {y}({self.assignments[y]})")
            if self.assignments[x] == self.assignments[y]:
                logger.debug("Split")
                self.propose_split(x, y)
            else:
                logger.debug("Merge")
                self.propose_merge(x, y)

    def propose_split(self, x, y, n_gibbs_iter=2):
        # Split cluster <old_cluster_id> into <old_cluster_id> and <new_cluster_id>
        old_cluster_id = self.assignments[x]
        new_cluster_id = np.max(self.assignments) + 1
        new_assignments = np.copy(self.assignments)
        
        # Assign each unit to one of x and y randomly   # TODO: random assignment or nearest?
        new_assignments[y] = new_cluster_id
        for i in range(self.N):
            if i != x and i != y and new_assignments[i] == old_cluster_id:
                new_assignments[i] = random.choice((old_cluster_id, new_cluster_id))
                #if self.log_p_mat[i][x] < self.log_p_mat[i][y]:
                #    new_assignments[i] = new_cluster_id
        logger.debug(f"\nCurrent state:\n{self.assignments}\nProposed state (init):\n{new_assignments}")
        
        # Re-assign each unit to one of the new clusters (= Gibbs sampling)
        self.gibbs_sampling(self.cluster_unit_ids(old_cluster_id),
                            (old_cluster_id, new_cluster_id),
                            new_assignments, n_iter=n_gibbs_iter)
        logger.debug(f"\nCurrent state:\n{self.assignments}\nProposed state (Gibbs):\n{new_assignments}")
        
        # Compare the probability of the current state and the proposed state
        p_current = self.log_prob_clustering()
        p_new = self.log_prob_clustering(new_assignments)
        logger.debug(f"Current state prob: {p_current}, Proposed state prob: {p_new}")
        if p_current < p_new:
            logger.debug("Accepted")
            self.assignments = new_assignments
        else:
            logger.debug("Rejected")
            
    def propose_merge(self, x, y):
        # Merge two clusters if the consensus sequences are same   # TODO: allow some diff when noise exists?
        if self.cluster_cons(self.assignments[x]) == self.cluster_cons(self.assignments[y]):
            logger.debug("Accepted")
            for i in range(self.N):
                if self.assignments[i] == old_cluster_id_y:
                    self.assignments[i] = old_cluster_id_x
        else:
            logger.debug("Rejected")
        
        """
        # Merge cluster <old_cluster_id_x> and <old_cluster_id_y> into <old_cluster_id_x>
        old_cluster_id_x = self.assignments[x]
        old_cluster_id_y = self.assignments[y]
        new_assignments = np.copy(self.assignments)
        
        # Change cluster assignment of the units in the cluster which units[y] belongs to
        for i in range(self.N):
            if new_assignments[i] == old_cluster_id_y:
                new_assignments[i] = old_cluster_id_x
        logger.debug(f"\nCurrent state:\n{self.assignments}\nProposed state:\n{new_assignments}")
        
        # Compare the probability of the current state and the proposed state
        p_current = self.log_prob_clustering()
        p_new = self.log_prob_clustering(new_assignments)
        logger.debug(f"Current state prob: {p_current}, Proposed state prob: {p_new}")
        if p_current < p_new:
            logger.debug("Accepted")
            self.assignments = new_assignments
        else:
            logger.debug("Rejected")
        """

In [12]:
def read_to_units(read):
    return [read.seq[unit.start:unit.end] for unit in read.units]

def read_to_quals(read):
    return [centromere_phreds[read.name][unit.start:unit.end] for unit in read.units]

In [14]:
read = copy(sync_reads[0])
smc = SplitMergeClustering(read_to_units(read), read_to_quals(read), alpha=1)

In [15]:
smc.log_prob_clustering()

-1936.4073285738723

In [16]:
smc.do_proposal()
smc.do_gibbs()

[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:197] Selected: 7(0) and 3(0)
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:199] Split
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:218] 
    Current state:
    [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
     0]
    Proposed state (init):
    [1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 1
     1]
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:176] Round 0
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:182] Unit 0: 1 -> 0
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:182] Unit 1: 0 -> 1
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:182] Unit 2: 0 -> 1
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:182] Unit 3: 1 -> 0
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:182] Unit 4: 0 -> 1
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:182] Unit 5: 0 -> 0
[D 190920 01:20:13 <ipython-input-12-a23e5296b461>:182] Unit 6: 0 -> 0
[D 190920 01:20:13 <

[D 190920 01:20:16 <ipython-input-12-a23e5296b461>:182] Unit 25: 2 -> 2
[D 190920 01:20:16 <ipython-input-12-a23e5296b461>:182] Unit 26: 2 -> 2
[D 190920 01:20:16 <ipython-input-12-a23e5296b461>:182] Unit 30: 2 -> 1
[D 190920 01:20:16 <ipython-input-12-a23e5296b461>:182] Unit 31: 2 -> 2
[D 190920 01:20:16 <ipython-input-12-a23e5296b461>:182] Unit 32: 2 -> 2
[D 190920 01:20:16 <ipython-input-12-a23e5296b461>:182] Unit 35: 2 -> 1
[D 190920 01:20:16 <ipython-input-12-a23e5296b461>:182] Unit 36: 2 -> 1
[D 190920 01:20:17 <ipython-input-12-a23e5296b461>:182] Unit 37: 1 -> 1
[D 190920 01:20:17 <ipython-input-12-a23e5296b461>:176] Round 1
[D 190920 01:20:17 <ipython-input-12-a23e5296b461>:182] Unit 1: 1 -> 2
[D 190920 01:20:17 <ipython-input-12-a23e5296b461>:182] Unit 2: 2 -> 2
[D 190920 01:20:17 <ipython-input-12-a23e5296b461>:182] Unit 4: 1 -> 1
[D 190920 01:20:17 <ipython-input-12-a23e5296b461>:182] Unit 8: 2 -> 1
[D 190920 01:20:17 <ipython-input-12-a23e5296b461>:182] Unit 9: 2 -> 2
[D 19

[D 190920 01:20:18 <ipython-input-12-a23e5296b461>:182] Unit 6: 3 -> 3
[D 190920 01:20:18 <ipython-input-12-a23e5296b461>:182] Unit 7: 3 -> 3
[D 190920 01:20:18 <ipython-input-12-a23e5296b461>:182] Unit 13: 3 -> 3
[D 190920 01:20:18 <ipython-input-12-a23e5296b461>:182] Unit 16: 0 -> 0
[D 190920 01:20:18 <ipython-input-12-a23e5296b461>:182] Unit 19: 0 -> 0
[D 190920 01:20:18 <ipython-input-12-a23e5296b461>:182] Unit 23: 0 -> 0
[D 190920 01:20:18 <ipython-input-12-a23e5296b461>:182] Unit 27: 3 -> 3
[D 190920 01:20:18 <ipython-input-12-a23e5296b461>:182] Unit 28: 0 -> 3
[D 190920 01:20:19 <ipython-input-12-a23e5296b461>:182] Unit 29: 3 -> 0
[D 190920 01:20:19 <ipython-input-12-a23e5296b461>:182] Unit 33: 0 -> 3
[D 190920 01:20:19 <ipython-input-12-a23e5296b461>:182] Unit 34: 3 -> 3
[D 190920 01:20:19 <ipython-input-12-a23e5296b461>:176] Round 1
[D 190920 01:20:19 <ipython-input-12-a23e5296b461>:182] Unit 0: 3 -> 0
[D 190920 01:20:19 <ipython-input-12-a23e5296b461>:182] Unit 3: 0 -> 0
[D 1

[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:125] Found in cache
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:125] Found in cache
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:229] Current state prob: -996.264047625724, Proposed state prob: -996.264047625724
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:234] Rejected
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:197] Selected: 23(4) and 37(1)
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:202] Merge
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:244] Rejected
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:197] Selected: 14(2) and 24(1)
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:202] Merge
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:244] Rejected
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:197] Selected: 8(1) and 24(1)
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:199] Split
[D 190920 01:20:20 <ipython-input-12-a23e5296b461>:218] 
    Current state:
    [0 2 2 4 1 4 3 3 1

[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 9: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 12: 2 -> 2
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 14: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 15: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 18: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 21: 2 -> 2
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 22: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 26: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 32: 2 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:176] Round 1
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 1: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 2: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 9: 6 -> 6
[D 190920 01:20:22 <ipython-input-12-a23e5296b461>:182] Unit 12: 2 -> 2
[D 1

[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 4: 1 -> 1
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 5: 4 -> 4
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 6: 3 -> 3
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 7: 3 -> 3
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 8: 1 -> 1
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 9: 6 -> 6
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 10: 5 -> 5
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 11: 5 -> 5
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 12: 2 -> 2
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 13: 0 -> 0
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 14: 6 -> 6
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 15: 6 -> 6
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 16: 4 -> 4
[D 190920 01:20:23 <ipython-input-12-a23e5296b461>:182] Unit 17: 1 -> 

In [24]:
smc.show_clustering()

Cluster 0 (5 units):
[8, 10, 11, 35, 36]
cagagtctgtttttccaaatttcggtcatcaaataatcatttattttgccacaacataaaaaataattgtctgaatatggaatgtcatacctctttgagctcgtaataaaatttccaatcaaactgtgttcaaaaatggaaattaaattttttggccatattttgcaattttgatgacccccctccttacaaaaaatgcgaaaattgatccaaaaattaatttcctaaatccttcaaaaagtaatagggatcgttagcactggtaattagctgctcaaaacagttattcttacatctatgtgaccatttttagccaagttataacgaaaatttcgtttgtaaatatcaacatttttg
=============================================================================================XX=========================================================================D========================================================================================================================X====================================================================
---
====X===========================================XX==================================X========XX===X=========X==========================================================D====DDDDD==DDDD=DD=D==================X===============================

In [25]:
c = ClusteringSeq([smc.cluster_cons(cluster_id) for cluster_id in np.unique(smc.assignments)], revcomp=False)
c.calc_dist_mat()
c.plot_dist_mat(variable_scale=True)

[I 190919 21:58:23 log:17] Starting distance matrix calculation 
[D 190919 21:58:23 seq:19] Computing row 0
[D 190919 21:58:23 seq:19] Computing row 8
[D 190919 21:58:23 seq:19] Computing row 1
[D 190919 21:58:23 seq:19] Computing row 7
[D 190919 21:58:23 seq:19] Computing row 2
[D 190919 21:58:23 seq:19] Computing row 6
[D 190919 21:58:23 seq:19] Computing row 3
[D 190919 21:58:23 seq:19] Computing row 5
[D 190919 21:58:23 seq:19] Computing row 4
[I 190919 21:58:23 log:19] Finished distance matrix calculation


In [17]:
for i in range(smc.N):
    read.units[i].id = smc.assignments[i]

In [18]:
v.show(read=read)

[D 190920 01:20:33 find_units:90] inners: {SelfAlignment(ab=354, ae=13802, bb=3, be=13447)}
[I 190920 01:20:33 log:17] Starting distance matrix calculation 
[D 190920 01:20:33 seq:19] Computing row 0
[D 190920 01:20:33 seq:19] Computing row 36
[D 190920 01:20:33 seq:19] Computing row 1
[D 190920 01:20:33 seq:19] Computing row 35
[D 190920 01:20:33 seq:19] Computing row 2
[D 190920 01:20:33 seq:19] Computing row 34
[D 190920 01:20:33 seq:19] Computing row 3
[D 190920 01:20:33 seq:19] Computing row 33
[D 190920 01:20:33 seq:19] Computing row 4
[D 190920 01:20:33 seq:19] Computing row 32
[D 190920 01:20:33 seq:19] Computing row 5
[D 190920 01:20:33 seq:19] Computing row 31
[D 190920 01:20:33 seq:19] Computing row 6
[D 190920 01:20:33 seq:19] Computing row 30
[D 190920 01:20:33 seq:19] Computing row 7
[D 190920 01:20:33 seq:19] Computing row 29
[D 190920 01:20:33 seq:19] Computing row 8
[D 190920 01:20:33 seq:19] Computing row 28
[D 190920 01:20:33 seq:19] Computing row 9
[D 190920 01:20:3

In [49]:
read = copy(sync_reads_by_id[198167])
smc = SplitMergeClustering(read_to_units(read), read_to_quals(read), alpha=1)

In [73]:
smc.do_proposal()
smc.do_gibbs()

[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 26(7) and 9(0)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 12(3) and 14(2)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 15(4) and 17(5)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 30(7) and 0(3)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 17(5) and 8(10)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ip

[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 23(7) and 22(0)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 23(7) and 19(6)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 27(8) and 17(5)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 0(3) and 18(8)
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 00:13:13 <ipython-input-11-a23e5296b461>:197] Selected: 3(1) and 17(5)
[D 190920 00:13:13 

[D 190920 00:13:14 <ipython-input-11-a23e5296b461>:191] [ 3  3  2  1  5  8  6  9 10  0  7  0  3  3  2  4  1  5  8  6  9  4  0  7
      4  0  7  8 10  0  7]


In [74]:
for i in range(smc.N):
    read.units[i].id = smc.assignments[i]

In [75]:
v.show(read=read)

[D 190920 00:13:18 find_units:90] inners: {SelfAlignment(ab=359, ae=11624, bb=0, be=11266)}
[I 190920 00:13:18 log:17] Starting distance matrix calculation 
[D 190920 00:13:18 seq:19] Computing row 0
[D 190920 00:13:18 seq:19] Computing row 29
[D 190920 00:13:18 seq:19] Computing row 1
[D 190920 00:13:18 seq:19] Computing row 28
[D 190920 00:13:18 seq:19] Computing row 2
[D 190920 00:13:18 seq:19] Computing row 27
[D 190920 00:13:18 seq:19] Computing row 3
[D 190920 00:13:18 seq:19] Computing row 26
[D 190920 00:13:18 seq:19] Computing row 4
[D 190920 00:13:18 seq:19] Computing row 25
[D 190920 00:13:18 seq:19] Computing row 5
[D 190920 00:13:18 seq:19] Computing row 24
[D 190920 00:13:18 seq:19] Computing row 6
[D 190920 00:13:18 seq:19] Computing row 23
[D 190920 00:13:18 seq:19] Computing row 7
[D 190920 00:13:18 seq:19] Computing row 22
[D 190920 00:13:18 seq:19] Computing row 8
[D 190920 00:13:18 seq:19] Computing row 21
[D 190920 00:13:18 seq:19] Computing row 9
[D 190920 00:13:1

## Clustering of units coming from two (adjacent) reads

In [13]:
units = read_to_units(sync_reads_by_id[73]) + read_to_units(sync_reads_by_id[198167])

In [14]:
quals = read_to_quals(sync_reads_by_id[73]) + read_to_quals(sync_reads_by_id[198167])

In [15]:
smc = SplitMergeClustering(units, quals, alpha=1)

In [28]:
smc.do_proposal(100)
smc.do_gibbs()

[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 32(2) and 22(13)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 2(12) and 61(17)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 59(20) and 6(1)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 44(6) and 40(14)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 9(22) and 53(12)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:

[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 39(11) and 28(10)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 3(8) and 66(13)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 31(5) and 38(11)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 15(20) and 39(11)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-inpu

[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 56(7) and 9(22)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 21(19) and 38(11)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 1(4) and 30(18)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 62(22) and 26(2)
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:202] Merge
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:244] Rejected
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:197] Selected: 54(16) and 59(20)
[D 190920 01

[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 23: 9 -> 9
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 24: 18 -> 18
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 25: 5 -> 5
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 26: 2 -> 2
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 27: 8 -> 8
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 28: 10 -> 10
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 29: 9 -> 9
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 30: 18 -> 18
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 31: 5 -> 5
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 32: 2 -> 2
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 33: 10 -> 10
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 34: 11 -> 11
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:182] Unit 35: 16 -> 16
[D 190920 01:32:04 <ipython-input-11-a23e5296b461>:1

[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 66: 13 -> 13
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 67: 9 -> 9
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 68: 18 -> 18
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:176] Round 2
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 0: 6 -> 6
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 1: 4 -> 4
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 2: 12 -> 12
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 3: 8 -> 8
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 4: 0 -> 0
[D 190920 01:32:05 <ipython-input-11-a23e5296b461>:182] Unit 5: 3 -> 3
[D 190920 01:32:06 <ipython-input-11-a23e5296b461>:182] Unit 6: 1 -> 1
[D 190920 01:32:06 <ipython-input-11-a23e5296b461>:182] Unit 7: 11 -> 11
[D 190920 01:32:06 <ipython-input-11-a23e5296b461>:182] Unit 8: 14 -> 14
[D 190920 01:32:06 <ipython-input-11-a23e5296b461>:182] Unit 9: 22 -> 2

In [35]:
smc.show_clustering()

Cluster 0 (2 units):
[4, 48]
cagagtctgtttttccaaatttcggtcatcaaataatcatttattttgccacaacataaaaaataattgtctgaacatggaatgtcatacctctctgagctcgtaataaaatttccaatcaaactgtgttcaaaaatggaaattaaatttttgggccatattatgcaaattttcatgacccccttcttacaaaaaatgcgaaaattgatccaaaaattaatttcctaaatccttcaaaaagtaatagggatcgttagcactggtaattagctgctcaaaacagatattcgtacatctatgtgaccatttttagccaagttataacgaaaatttcgtttgtaaatatcaacatttttg
===========================================================================X=================X==========================================================X=========I==D========X=========X==D=================================================================================================X====I==D==================================================================
---
Cluster 1 (2 units):
[6, 50]
cagaatctgtttttccaaatttcggtcatcaaataatcatttattttgccacaacataaaaaataattgtctgaatatggaatgtcatacctcactgagctcgtaataaaatttccaattaaactgtgttcaaaaatggaaacaaaattttttggcattattttgcaaattttgatgaccaccctccttacaaaaaatgcgaaaagtgattcaaaaatt

In [36]:
PairwiseAlignment("cagagtctgtttttccaaatttcggtcatcaaaaatcatttattttgccacaacataaaaataattgtctgaatatggaatgtcatacctcactgagctcgtaataaaatttccaatcaaactgtgttcaaaaatggaaattaaattttttggccatatttggcaaattttgatgaccccctccttacaaaaaatgcgaaaattggtccaaaaattaatttcctaaatccttcaaaaagtaatagggatcgttagcactggtaatttgctgctcaaaacagatattcgtacatctatgtgaccattgctagccaagttataacgaaaatttcgtttgtaaatatcaacatttttg", "cagagtctgtttttccaaatttcggtcatcaaaaaatcatttattttgccacaacataaaaaataattgtctgaatatggaatgtcatacctcactgagctcgtaataaaatttccaatcaaactgtgttcaaaaatggaaattaaattttttggccatatttggcaaattttgatgacccccctccttacaaaaaatgcgaaaattggtccaaaaattaatttcctaaatccttcaaaaagtaatagggatcgttagcactggtaatttgctgctcaaaacagatattcgtacatctatgtgaccattgctagccaagttataacgaaaatttcgtttgtaaatatcaacatttttg").show()

                                   -                          -                                                                                                                        -                                                                                                                                                                              
cagagtctgtttttccaaatttcggtcatcaaaaa tcatttattttgccacaacataaaaa taattgtctgaatatggaatgtcatacctcactgagctcgtaataaaatttccaatcaaactgtgttcaaaaatggaaattaaattttttggccatatttggcaaattttgatgaccccc tccttacaaaaaatgcgaaaattggtccaaaaattaatttcctaaatccttcaaaaagtaatagggatcgttagcactggtaatttgctgctcaaaacagatattcgtacatctatgtgaccattgctagccaagttataacgaaaatttcgtttgtaaatatcaacatttttg
                                   a                          a                                                                                                                        c                                                                                                  

In [29]:
logzero.loglevel(logging.INFO)

In [30]:
c = ClusteringSeq([smc.cluster_cons(cluster_id) for cluster_id in np.unique(smc.assignments)], revcomp=False)
c.calc_dist_mat()
c.plot_dist_mat(variable_scale=True)

[I 190920 01:32:12 log:17] Starting distance matrix calculation 
[I 190920 01:32:12 log:19] Finished distance matrix calculation


In [31]:
read_a = sync_reads_by_id[73]
for i in range(len(read_a.units)):
    read_a.units[i].id = smc.assignments[i]

In [32]:
read_b = sync_reads_by_id[198167]
for i in range(len(read_b.units)):
    read_b.units[i].id = smc.assignments[-(len(read_b.units) - i)]

In [33]:
v.show(read=read_a)

[I 190920 01:32:20 log:17] Starting distance matrix calculation 
[I 190920 01:32:20 log:19] Finished distance matrix calculation


In [34]:
v.show(read=read_b)

[I 190920 01:32:22 log:17] Starting distance matrix calculation 
[I 190920 01:32:22 log:19] Finished distance matrix calculation


In [42]:
logzero.loglevel(logging.DEBUG)

## Conclusion

同じ領域から読まれてきたリードを集めることができたら (+ 計算量が許せば) うまくいきそう。